In [112]:
import pandas as pd
import numpy as np

# Default configuration for time periods in traffic data
input_file = "outputs/2025-10-08/extra_lane_ML_new_trigger/model_run.csv"
run_folder = 'test_get_toll/test_3_lanes_ML_new_trigger'

# Here we load th value of the counts and we multiply the peak hour values by a constant
lights_w = 1

heavies_w = 3
heavies_w_toll = 3
heavies_w_vot = 3

medium_A_w = 3 # TBD: Maybe try 2.5 or 2.75 for every pce value
medium_A_w_toll = 3
medium_A_w_vot = 3

medium_B_w = 3
medium_B_w_toll = 4
medium_B_w_vot = 3

heavy_A_w = 3
heavy_A_w_toll = 6
heavy_A_w_vot = 3

heavy_B_w = 3
heavy_B_w_toll = 3
heavy_B_w_vot = 3

traffic_condition = 1500
speed_condition = 55

# Default time periods list (for reference)
default_time_periods = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

# Create the base scenario: hour -> time period mapping
hour_to_period = {
    0: "Night",
    1: "Night",
    2: "Night",
    3: "Night",
    4: "Night",
    5: "AM-Early",
    6: "AM-Peak",
    7: "AM-Shoulder",
    8: "AM-Shoulder",
    9: "AM-Shoulder",
    10: "MD",
    11: "MD",
    12: "MD",
    13: "MD",
    14: "PM-Shoulder",
    15: "PM-Shoulder",
    16: "PM-Peak",
    17: "PM-Peak",
    18: "PM-Late",
    19: "PM-Late",
    20: "PM-Late",
    21: "PM-Late",
    22: "PM-Late",
    23: "Night"
}

# Define the segments and their parameters

awt_adt = 1.1 # Average weekday traffic (AWT) to average daily traffic (ADT) ratio
peak_factor = 1.05 # Peak factor for adjustment at peak hour traffic

hov_percentage = pd.DataFrame({
    'Year' : [2032,2040,2050],
    'HOV percentage' : [0,0,0]
})

hov_percentage.set_index('Year', inplace=True)

# Define segment parameters base
seg_params = pd.DataFrame({
    'SegDir':   ["1NB","1SB","2NB","2SB","3NB","3SB","4NB","4SB","5NB","5SB","6NB","6SB","7NB","7SB","8NB","8SB","9NB","9SB","10NB","10SB"],
    'Length':    [0.7,0.7,0.7,0.7,0.5,0.5,1.6,1.6,2,2,3.6,3.6,2.9,2.9,3.8,3.8,3.4,3.4,4.5,4.5],
    'Inscope':   [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8],
    'Lanes_GP':  [4]*20, #
    'Lanes_ML':  [3]*20, # Lanes_ML': [2,2,2,2,2,2,2,2,3,3,2,2,2,2], # Do test changing segment 5
    'CapPerLane_GP': [2000]*20,
    'CapPerLane_ML': [1800]*20,
    'Speed_GP':  [55]*6 + [65]*2 + [70]*12,
    'Speed_ML':  [70]*20,
    'Alpha_GP':  [1]*20,
    'Beta_GP':   [6]*20,
    'Alpha_ML':  [1]*20,
    'Beta_ML':   [5]*20,
    'Min_Toll_2016': [None]*20,
    'Max_Toll_2016': [None]*20,
    'LanesGP_AM_Peak': [5]*20,
    'LanesGP_PM_Peak': [5]*20,
})

seg_params.set_index('SegDir', inplace=True)

# Compute capacities as lanes * cap per lane
seg_params['Cap_GP'] = seg_params['Lanes_GP'] * seg_params['CapPerLane_GP']
seg_params['Cap_ML'] = seg_params['Lanes_ML'] * seg_params['CapPerLane_ML']

# Compute peak capacities as Alpha * base capacity
seg_params['CapGP_Peak'] = seg_params['Alpha_GP'] * seg_params['Cap_GP']
seg_params['CapML_Peak'] = seg_params['Alpha_ML'] * seg_params['Cap_ML']

# Optional: if you want integer capacities
seg_params[['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']] = seg_params[
    ['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']
].astype(int)

# Preview
seg_params

,Length,Inscope,Lanes_GP,Lanes_ML,CapPerLane_GP,CapPerLane_ML,Speed_GP,Speed_ML,Alpha_GP,Beta_GP,Alpha_ML,Beta_ML,Min_Toll_2016,Max_Toll_2016,LanesGP_AM_Peak,LanesGP_PM_Peak,Cap_GP,Cap_ML,CapGP_Peak,CapML_Peak
SegDir,,,,,,,,,,,,,,,,,,,,
1NB,0.7,0.8,4,3,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
1SB,0.7,0.8,4,3,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
2NB,0.7,0.8,4,3,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
2SB,0.7,0.8,4,3,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
3NB,0.5,0.8,4,3,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
3SB,0.5,0.8,4,3,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
4NB,1.6,0.8,4,3,2000,1800,65,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
4SB,1.6,0.8,4,3,2000,1800,65,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400
5NB,2.0,0.8,4,3,2000,1800,70,70,1,6,1,5,None,None,5,5,8000,5400,8000,5400


In [113]:
lookup_period_file = r"inputs/LookUp_Period.csv"

lookup_period = pd.read_csv(
    lookup_period_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Clip y reasignar
lookup_period = lookup_period * 0

lookup_period


,Bonus/Mile,4 Periods
Period,,
Night,0.0,
AM-Early,0.0,
AM-Peak,0.0,
AM-Shoulder,0.0,
MD,0.0,
PM-Shoulder,0.0,
PM-Peak,0.0,
PM-Late,0.0,


In [114]:
def get_vot(row):

    max_VC = 1.2  # TBD: check if we need to change this value
    ETC_discount = 0.15
    
    captureRateLights =  row["CaptureRateLights"]
    captureRateMediumA =  row["CaptureRateMediumA"]
    captureRateMediumB =  row["CaptureRateMediumB"]
    captureRateHeavyA =  row["CaptureRateHeavyA"]

    tollLights = row["TollLights"]
    tollMediumA = row["TollMediumA"]
    tollMediumB = row["TollMediumB"]
    tollHeavyA = row["TollHeavyA"]

    # ml_pce = seg_params.loc[row["SegDir"], 'Inscope'] * (
    #     row["TotalLights"] * lights_w * captureRateLights + 
    #     row["TotalMediumA"] * medium_A_w * captureRateMediumA +
    #     row["TotalMediumB"] * medium_B_w * captureRateMediumB +
    #     row["TotalHeavyA"] * heavy_A_w * captureRateHeavyA
    # )

    ml_pce = (
        row["InScopeLights"] * lights_w * captureRateLights + 
        row["InScopeMediumA"] * medium_A_w * captureRateMediumA +
        row["InScopeMediumB"] * medium_B_w * captureRateMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureRateHeavyA
    )
    
    gp_pce = np.min(np.array([row["Corridor PCE"] - ml_pce, row["Suppressed ratio ref"] * seg_params['Cap_GP'][row['SegDir']]]))
    
    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((ml_pce + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    timeML = 60 * row["Length"] / speedML

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    timeSavings = timeML - timeGP

    gp_vc = gp_pce / row["Capacity GP"]

    ml_vc = (ml_pce + row["HOV3"]) / row["Capacity ML"]

    # bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"])) 

    # bonusAux = (2.5 if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else 1.6)

    bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"]))

    bonusAux = (2.5 if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else 1.6) 

    bonusSeg = (lookup_period.loc[row["Period"], "Bonus/Mile"] + (bonusAux if gp_vc > 0.3 else 0) * (gp_vc - ml_vc)) * row["Length"] # TBD: add condition for gp_vc

    votLights = -60 * (1 - ETC_discount) * row["Length"] * tollLights / (timeSavings - bonusSeg - bonusRel) # TBD: add household income effect

    votMediumA = -60 * (1 - ETC_discount) * row["Length"] * tollMediumA / (timeSavings - bonusSeg - bonusRel)

    votMediumB = -60 * (1 - ETC_discount) * row["Length"] * tollMediumB / (timeSavings - bonusSeg - bonusRel)

    votHeavyA = -60 * (1 - ETC_discount) * row["Length"] * tollHeavyA / (timeSavings - bonusSeg - bonusRel)

    return pd.Series([votLights, votMediumA, votMediumB, votHeavyA, speedGP, speedML, bonusSeg, timeML, timeGP, timeSavings, bonusRel], index=["VOT Lights", "VOT MediumA", "VOT MediumB", "VOT HeavyA", "Speed GP", "Speed ML", "Bounus Seg", "Time ML", "Time GP", "Time Savings", "BonusRel"])

In [115]:
from scipy.stats import lognorm
from scipy.optimize import minimize, least_squares, Bounds
import numpy as np

ETC_discount = 0.15
max_VC = 1.2  # TBD: check if we need to change this value

def objective_integrated(x, row, tollLights):

    captureRateLights, captureRateMediumA, captureRateMediumB, captureRateHeavyA = x

    tollMediumA = tollLights * medium_A_w_toll
    tollMediumB = tollLights * medium_B_w_toll
    tollHeavyA = tollLights * heavy_A_w_toll

    ml_pce = (
        row["InScopeLights"] * lights_w * captureRateLights + 
        row["InScopeMediumA"] * medium_A_w * captureRateMediumA +
        row["InScopeMediumB"] * medium_B_w * captureRateMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureRateHeavyA
    )

    gp_pce = np.min(np.array([row["Corridor PCE"] - ml_pce, row["Suppressed ratio ref"] * seg_params['Cap_GP'][row['SegDir']]]))
    
    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((ml_pce + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    timeML = 60 * row["Length"] / speedML

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    timeSavings = timeML - timeGP

    gp_vc = gp_pce / row["Capacity GP"]

    ml_vc = (ml_pce + row["HOV3"]) / row["Capacity ML"]

    bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"]))

    bonusAux = (2.5 if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else 1.6)

    bonusSeg = (lookup_period.loc[row["Period"], "Bonus/Mile"] + (bonusAux if gp_vc > 0.3 else 0) * (gp_vc - ml_vc)) * row["Length"] # TBD: add condition for gp_vc

    votLights = -60 * (1 - ETC_discount) * row["Length"] * tollLights / (timeSavings - bonusSeg - bonusRel) # TBD: add household income effect

    votMediumA = -60 * (1 - ETC_discount) * row["Length"] * tollMediumA / (timeSavings - bonusSeg - bonusRel)

    votMediumB = -60 * (1 - ETC_discount) * row["Length"] * tollMediumB / (timeSavings - bonusSeg - bonusRel)

    votHeavyA = -60 * (1 - ETC_discount) * row["Length"] * tollHeavyA / (timeSavings - bonusSeg - bonusRel)

    # TBD: Integrate Reliability
    # Here we compute the lognormal cumulative function for the light vehicles

    mu_lights = row["B1"]

    mu_heavies = mu_lights

    std_dev = row["B2"]

    scale_lights = np.exp(mu_lights)

    scale_heavies = np.exp(mu_heavies) * heavies_w_vot # TBD: change heavies betas (ask Borja)

    # We create a lognormal distribution object

    dist_lights = lognorm(s=std_dev, scale=scale_lights)

    dist_heavies = lognorm(s=std_dev, scale=scale_heavies)

    calcCaptureRateLights = 1 - dist_lights.cdf(votLights)

    calcCaptureMediumA = 1 - dist_heavies.cdf(votMediumA)

    calcCaptureMediumB = 1 - dist_heavies.cdf(votMediumB)

    calcCaptureHeavyA = 1 - dist_heavies.cdf(votHeavyA)

    convergenceLights = calcCaptureRateLights - captureRateLights

    convergenceMediumA = calcCaptureMediumA - captureRateMediumA

    convergenceMediumB = calcCaptureMediumB - captureRateMediumB

    convergenceHeavyA = calcCaptureHeavyA - captureRateHeavyA

    return convergenceLights ** 2 + convergenceMediumA ** 2 + convergenceMediumB ** 2 + convergenceHeavyA ** 2

def optimize_capture(row, M, toll_value):

    capture_first_guess = row["CaptureRateLights"]
    max_capture = row["MaxCapture"]

    ub = [
        max_capture,
        max_capture,
        max_capture,
        max_capture
    ]

    lb = [
        row["MinCapture"],
        row["MinCapture"],
        row["MinCapture"],
        row["MinCapture"]
    ]

    bounds = Bounds(lb=lb, ub=ub)

    x0 = [
        capture_first_guess,
        capture_first_guess,
        capture_first_guess,
        capture_first_guess
    ] # Initial guess, TBD

    Max_iter = 10000000

    # Solve
    result = minimize(
        objective_integrated,
        x0,
        method='trust-constr',
        args=(row,toll_value),
        bounds=bounds,
        # constraints=[nlc],
        options={
            "verbose": 0,
            "maxiter": Max_iter,
            "gtol": 1e-6,
            "xtol": 1e-6,
            "barrier_tol": 1e-6,
            "initial_tr_radius": 1.0,
            "initial_constr_penalty": 1.0,
            "sparse_jacobian": True
        }
    )

    x_opt = result.x
    z_opt = result.fun
    x_opt = np.minimum(bounds.ub, np.maximum(bounds.lb, result.x))
    return x_opt

In [116]:
import pandas as pd
import sys

first_model_df = pd.read_csv(f"{input_file}")


first_model_df = first_model_df[first_model_df["Year"] == 2050].reset_index()

def optimize_row(row):

    # We get the value of the toll for the highest value of revenue

    captureLights = row["CaptureRateLights"]

    captureMediumA = row["CaptureRateMediumA"]

    captureMediumB = row["CaptureRateMediumB"]

    captureHeavyA = row["CaptureRateHeavyA"]

    ml_pce = (row["InScopeLights"] * captureLights + row["HOV3"] + 
              row["InScopeMediumA"] *  captureMediumA * medium_A_w + 
              row["InScopeMediumB"] *  captureMediumB * medium_B_w + 
              row["InScopeHeavyA"] *  captureHeavyA * heavy_A_w)
    
    traffic_lane =  ml_pce/ seg_params.loc[row["SegDir"], 'Lanes_ML']
    # pce_speed = seg_params.loc[row["SegDir"], 'Inscope'] * (row["TotalLights"] * lights_w * captureLights + 
    #                                                         row["TotalMediumA"] * medium_A_w * captureMediumA +
    #                                                         row["TotalMediumB"] * medium_B_w * captureMediumB +
    #                                                         row["TotalHeavyA"] * heavy_A_w * captureHeavyA
    #                                                         )

    pce_speed = (
        row["InScopeLights"] * lights_w * captureLights + 
        row["InScopeMediumA"] * medium_A_w * captureMediumA +
        row["InScopeMediumB"] * medium_B_w * captureMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureHeavyA
    )

    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((pce_speed + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    toll = row["TollLights"]

    changed = False

    # print(f'Segment: {row["SegDir"]}, Period: {row["Period"]}, toll: {toll}, traffic: {traffic_lane}, speed: {speedML}')

    while (traffic_lane > 1500) and (speedML < 55):
        changed = True
        x_opt = optimize_capture(row, 1, toll)
        captureLights, captureMediumA, captureMediumB, captureHeavyA = x_opt
        ml_pce = (row["InScopeLights"] * captureLights + row["HOV3"] + 
              row["InScopeMediumA"] *  captureMediumA * medium_A_w + 
              row["InScopeMediumB"] *  captureMediumB * medium_B_w + 
              row["InScopeHeavyA"] *  captureHeavyA * heavy_A_w)
        
        pce_speed = (
            row["InScopeLights"] * lights_w * captureLights + 
            row["InScopeMediumA"] * medium_A_w * captureMediumA +
            row["InScopeMediumB"] * medium_B_w * captureMediumB +
            row["InScopeHeavyA"] * heavy_A_w * captureHeavyA
        )

        traffic_lane =  ml_pce/ seg_params.loc[row["SegDir"], 'Lanes_ML']
        speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((pce_speed + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests
        # print(f'Segment: {row["SegDir"]}, Period: {row["Period"]}, toll: {toll}, traffic: {traffic_lane}, speed: {speedML}')
        objective_integrated(x_opt, row, toll)
        toll += 0.05

        if toll > 20:
            break

    if changed:
        changed = False
        print(f'Segment: {row["SegDir"]}, Period: {row["Period"]}, toll: {toll}, traffic: {traffic_lane}, speed: {speedML}')

    return pd.Series([captureLights, captureMediumA, captureMediumB, captureHeavyA, toll], 
                index=["CaptureRateLights", "CaptureRateMediumA", "CaptureRateMediumB", "CaptureRateHeavyA", "TollLights"])

first_model_df[["CaptureRateLights", "CaptureRateMediumA", "CaptureRateMediumB", "CaptureRateHeavyA", "TollLights"]] = first_model_df.apply(
    optimize_row, axis=1, result_type='expand'
)

C:\Users\crisp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\scipy\optimize\_differentiable_functions.py:376: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


Segment: 2SB, Period: PM-Shoulder, toll: 10.049999947113973, traffic: 1499.0953796637841, speed: 49.97618239718373
Segment: 2SB, Period: PM-Peak, toll: 9.549993304950565, traffic: 1499.8028263308543, speed: 49.94244100869314


In [117]:
first_model_df["TollMediumA"] = first_model_df.apply(
    lambda row: row["TollLights"] * medium_A_w_toll,
    axis=1
)

first_model_df["TollMediumB"] = first_model_df.apply(
    lambda row: row["TollLights"] * medium_B_w_toll,
    axis=1
)

first_model_df["TollHeavyA"] = first_model_df.apply(
    lambda row: row["TollLights"] * heavy_A_w_vot,
    axis=1
)
first_model_df[["VOT Lights", "VOT MediumA", "VOT MediumB", "VOT HeavyA", "Speed GP", "Speed ML", "Bounus Seg", "Time ML", "Time GP", "Time Savings", "BonusRel"]] = first_model_df.apply(
    get_vot, axis=1, result_type='expand'
)

first_model_df

,level_0,Unnamed: 0,index,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,...,RevenuePerDayLights,RevenuePerDayMediumA,RevenuePerDayMediumB,RevenuePerDayHeavyA,TransactionsDay_Lights,TransactionsDay_MediumA,TransactionsDay_MediumB,TransactionsDay_HeavyA,TransactionsDay,TotalVeh_hours
0,0,0,320,2050,1NB,1,NB,Night,6,OP,...,1056.356666,109.791456,16.164208,819.597371,1762.180689,61.050208,8.988206,455.742114,2287.961217,12063.600
1,1,1,321,2050,1NB,1,NB,AM-Early,1,OP,...,1256.488103,56.361628,37.032438,235.126430,877.353196,13.118324,8.619402,54.726324,953.817246,7845.500
2,2,2,322,2050,1NB,1,NB,AM-Peak,1,Peak,...,3164.648949,271.685280,142.361278,462.584213,1755.719311,50.242865,26.326927,85.545880,1917.834983,10361.925
3,3,3,323,2050,1NB,1,NB,AM-Shoulder,3,OP,...,8054.366397,930.391455,232.395000,1878.727613,4468.487911,172.057522,42.976865,347.433562,5030.955860,21060.300
4,4,4,324,2050,1NB,1,NB,MD,4,OP,...,9804.075738,1162.996882,328.696078,3368.762155,5439.154380,215.071084,60.785220,622.979596,6337.990281,26340.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,155,155,475,2050,10SB,10,SB,AM-Shoulder,3,OP,...,20089.777714,3262.685568,973.044584,7133.649123,1871.545018,101.316252,30.215976,221.521374,2224.598619,19640.400
156,156,156,476,2050,10SB,10,SB,MD,4,OP,...,24478.600753,3145.842373,1179.040112,10284.206006,2510.084441,107.526979,40.300373,351.520984,3009.432778,26562.400
157,157,157,477,2050,10SB,10,SB,PM-Shoulder,2,OP,...,33874.379001,3182.308589,1054.009535,7764.038171,2923.355534,91.544305,30.320306,223.345240,3268.565385,18819.800
158,158,158,478,2050,10SB,10,SB,PM-Peak,2,Peak,...,41588.253635,3002.830113,653.673683,6935.567150,3774.379200,90.841512,19.774913,209.814536,4094.810161,22921.290


In [118]:
first_model_df["MLVeh_Lights"] = first_model_df.apply(
    lambda row: row["InScopeLights"] * row["CaptureRateLights"],
    axis=1
)

first_model_df["GPVeh_Lights"] = first_model_df.apply(
    lambda row: row["TotalLights"] - row["MLVeh_Lights"] - row["HOV3"],
    axis=1
)

first_model_df["MLVeh_MediumA"] = first_model_df.apply(
    lambda row: row["InScopeMediumA"] * row["CaptureRateMediumA"],
    axis=1
)

first_model_df["GPVeh_MediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] - row["MLVeh_MediumA"],
    axis=1
)

first_model_df["MLVeh_MediumB"] = first_model_df.apply(
    lambda row: row["InScopeMediumB"] * row["CaptureRateMediumB"],
    axis=1
)

first_model_df["GPVeh_MediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] - row["MLVeh_MediumB"],
    axis=1
)

first_model_df["MLVeh_HeavyA"] = first_model_df.apply(
    lambda row: row["InScopeHeavyA"] * row["CaptureRateHeavyA"],
    axis=1
)

first_model_df["GPVeh_HeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] - row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["MLVeh"] = first_model_df.apply(
    lambda row: row["MLVeh_Lights"] + row["MLVeh_MediumA"] + row["MLVeh_MediumB"] + row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["GPVeh"] = first_model_df.apply(
    lambda row: row["GPVeh_Lights"] + row["GPVeh_MediumA"] + row["GPVeh_MediumB"] + row["GPVeh_HeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["GPVehDay"] = first_model_df.apply(
    lambda row: row["GPVeh"] * row["Hours/Day"],
    axis=1
)

first_model_df["ML PCE"] = first_model_df.apply(
    lambda row:row["MLVeh_Lights"] * lights_w
    + row["MLVeh_MediumA"] * medium_A_w
    + row["MLVeh_MediumB"] * medium_B_w
    + row["MLVeh_HeavyA"] * heavy_A_w, # TBD: is HOV included in the PCE?
    axis=1
)

first_model_df["Traffic per Lane"] = first_model_df.apply(
    lambda row: row["ML PCE"] / seg_params.loc[row["SegDir"], 'Lanes_ML'], # TBD: is HOV included in the PCE?
    axis=1
)

first_model_df["GP PCE"] = first_model_df.apply(
    lambda row:row["GPVeh_Lights"] * lights_w
    + row["GPVeh_MediumA"] * medium_A_w
    + row["GPVeh_MediumB"] * medium_B_w
    + row["GPVeh_HeavyA"] * heavy_A_w
    + row["TotalHeavyB"] * heavy_B_w, # TBD: is HOV included in the PCE?
    axis=1
)

# We add the TollPerSeg column

first_model_df["TollLightsPerSeg"] = first_model_df.apply(
    lambda row: row["TollLights"] * row["Length"],
    axis=1
)

first_model_df["TollMediumAPerSeg"] = first_model_df.apply(
    lambda row: row["TollMediumA"] * row["Length"],
    axis=1
)

first_model_df["TollMediumBPerSeg"] = first_model_df.apply(
    lambda row: row["TollMediumB"] * row["Length"],
    axis=1
)

first_model_df["TollHeavyAPerSeg"] = first_model_df.apply(
    lambda row: row["TollHeavyA"] * row["Length"],
    axis=1
)

# Toll blend
first_model_df["TollBlend"] = first_model_df.apply(
    lambda row: (row["TollLights"] * row["MLVeh_Lights"] 
                + row["TollMediumA"] * row["MLVeh_MediumA"]
                + row["TollMediumB"] * row["MLVeh_MediumB"]
                + row["TollHeavyA"] * row["MLVeh_HeavyA"]
                ) / (row["MLVeh"]),
    axis=1
)

first_model_df["RevenuePerHour"] = first_model_df.apply(
    lambda row: row["TollLightsPerSeg"] * row["MLVeh_Lights"]
    + row["TollMediumAPerSeg"] * row["MLVeh_MediumA"]
    + row["TollMediumBPerSeg"] * row["MLVeh_MediumB"]
    + row["TollHeavyAPerSeg"] * row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["RevenuePerHourLights"] = first_model_df.apply(
    lambda row: row["TollLightsPerSeg"] * row["MLVeh_Lights"],
    axis=1
)

first_model_df["RevenuePerHourMediumA"] = first_model_df.apply(
    lambda row: row["TollMediumAPerSeg"] * row["MLVeh_MediumA"],
    axis=1
)

first_model_df["RevenuePerHourMediumB"] = first_model_df.apply(
    lambda row: row["TollMediumBPerSeg"] * row["MLVeh_MediumB"],
    axis=1
)

first_model_df["RevenuePerHourHeavyA"] = first_model_df.apply(
    lambda row: row["TollHeavyAPerSeg"] * row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["RevenuePerDay"] = first_model_df.apply(
    lambda row: row["RevenuePerHour"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayLights"] = first_model_df.apply(
    lambda row: row["RevenuePerHourLights"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayMediumA"] = first_model_df.apply(
    lambda row: row["RevenuePerHourMediumA"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayMediumB"] = first_model_df.apply(
    lambda row: row["RevenuePerHourMediumB"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayHeavyA"] = first_model_df.apply(
    lambda row: row["RevenuePerHourHeavyA"] * row["Hours/Day"],
    axis=1
)

first_model_df["TransactionsDay_Lights"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_Lights"],
    axis=1
)

first_model_df["TransactionsDay_MediumA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_MediumA"],
    axis=1
)

first_model_df["TransactionsDay_MediumB"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_MediumB"],
    axis=1
)

first_model_df["TransactionsDay_HeavyA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["TransactionsDay"] = first_model_df.apply(
    lambda row: row["TransactionsDay_Lights"] + row["TransactionsDay_MediumA"] + row["TransactionsDay_MediumB"] + row["TransactionsDay_HeavyA"],
    axis=1
)

first_model_df["TotalVeh_hours"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["TotalVeh"],
    axis=1
)

first_model_df.to_csv(f'{run_folder}/test_model_run.csv')

first_model_df

,level_0,Unnamed: 0,index,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,...,RevenuePerDayLights,RevenuePerDayMediumA,RevenuePerDayMediumB,RevenuePerDayHeavyA,TransactionsDay_Lights,TransactionsDay_MediumA,TransactionsDay_MediumB,TransactionsDay_HeavyA,TransactionsDay,TotalVeh_hours
0,0,0,320,2050,1NB,1,NB,Night,6,OP,...,1056.356666,109.791456,21.552277,819.597371,1762.180689,61.050208,8.988206,455.742114,2287.961217,12063.600
1,1,1,321,2050,1NB,1,NB,AM-Early,1,OP,...,1256.488103,56.361628,49.376584,235.126430,877.353196,13.118324,8.619402,54.726324,953.817246,7845.500
2,2,2,322,2050,1NB,1,NB,AM-Peak,1,Peak,...,3164.648949,271.685280,189.815038,462.584213,1755.719311,50.242865,26.326927,85.545880,1917.834983,10361.925
3,3,3,323,2050,1NB,1,NB,AM-Shoulder,3,OP,...,8054.366397,930.391455,309.859999,1878.727613,4468.487911,172.057522,42.976865,347.433562,5030.955860,21060.300
4,4,4,324,2050,1NB,1,NB,MD,4,OP,...,9804.075738,1162.996882,438.261437,3368.762155,5439.154380,215.071084,60.785220,622.979596,6337.990281,26340.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,155,155,475,2050,10SB,10,SB,AM-Shoulder,3,OP,...,20089.777714,3262.685568,1297.392779,7133.649123,1871.545018,101.316252,30.215976,221.521374,2224.598619,19640.400
156,156,156,476,2050,10SB,10,SB,MD,4,OP,...,24478.600753,3145.842373,1572.053482,10284.206006,2510.084441,107.526979,40.300373,351.520984,3009.432778,26562.400
157,157,157,477,2050,10SB,10,SB,PM-Shoulder,2,OP,...,33874.379001,3182.308589,1405.346047,7764.038171,2923.355534,91.544305,30.320306,223.345240,3268.565385,18819.800
158,158,158,478,2050,10SB,10,SB,PM-Peak,2,Peak,...,41588.253635,3002.830113,871.564911,6935.567150,3774.379200,90.841512,19.774913,209.814536,4094.810161,22921.290


In [119]:
period_order = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

first_model_df["Period"] = pd.Categorical(
    first_model_df["Period"],
    categories=period_order,
    ordered=True
)

sum_GPVeh = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="GPVeh"
)

sum_MLVeh = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="MLVeh"
)

sumRevHour = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="RevenuePerHour"
)

sumTollLights = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="TollLights"
)

sumCaptureRateLights = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="CaptureRateLights"
)

revenuesDay = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="RevenuePerDay"
)

speedML = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="Speed ML"
)

trafficPerLane = first_model_df.pivot(
    index=["Year", "Direction", "Segment"],
    columns="Period",
    values="Traffic per Lane"
)

pivot_vals = pd.concat([sum_GPVeh, sum_MLVeh, sumRevHour, sumTollLights, sumCaptureRateLights, revenuesDay], axis=1, keys=["GPVeh", "MLVeh", "RevenuePerHour", "TollLights", "CaptureRateLights","RevenueDay"])

# pivot_vals.to_csv(f'I24_study/{today_str}/pivot.csv')


pivot_vals.to_csv(f'{run_folder}/pivot_test.csv')

speedML.to_csv(f'{run_folder}/speed.csv')

trafficPerLane.to_csv(f'{run_folder}/traffic.csv')

revenuesDay.to_csv(f'{run_folder}/revenues_test.csv')

In [120]:
def generate_revenue_stream(final_df, file_name):

    output_df = pd.DataFrame(columns=['Year', 'Total Revenue', 'Total Transactions', 'AADT', 'Toll/AADT/mi', 'Capture']) # AADT refers to ML

    years = [2032, 2040, 2050]

    Anualization_factor = 295 # TBD: Just changed from 290, check if it makes sense
    traffic_anualization = 310 # TBD: Check if it fits
    year_days = 365

    length_correction_factor = 1.055

    for year in years:

        yearly_df = final_df[final_df["Year"] == year]

        # We compute the revenues

        revenues_df = yearly_df.groupby(['Segment', 'Direction'])[["RevenuePerDay"]].sum().reset_index() #

        revenues_df["Annual Revenue"] = revenues_df["RevenuePerDay"] * Anualization_factor # We have to change 290 for an annual constant

        total_revenues = revenues_df["Annual Revenue"].sum() * length_correction_factor # TBD: Change 1.02 for a parameter

        # We compute the total transactions

        transactions_df = yearly_df.groupby(['Segment', 'Direction'])[["TransactionsDay"]].sum().reset_index()

        # transactions_df_aux.append(transactions_df)

        total_transactions = transactions_df["TransactionsDay"].sum() * traffic_anualization # TBD: Change 315 for a parameter

        # We compute the AADT

        aadt_df = yearly_df.groupby(['SegDir'])[["TransactionsDay"]].sum()

        gp_traffic_df = yearly_df.groupby(['SegDir'])[["GPVehDay"]].sum()

        total_df = yearly_df.groupby(['SegDir'])[["TotalVeh_hours"]].sum()

        df_lengths = pd.DataFrame(seg_params["Length"])

        merged_df = pd.merge(aadt_df, df_lengths, on='SegDir')

        merged_gp_traffic_df = pd.merge(gp_traffic_df, df_lengths, on='SegDir')

        # Compute product
        merged_df['Weighted'] = merged_df['Length'].values * merged_df["TransactionsDay"].values

        merged_gp_traffic_df['Weighted'] = merged_gp_traffic_df['Length'] * merged_gp_traffic_df["GPVehDay"]

        # Sum weighted values
        total_aadt = (traffic_anualization/year_days) * merged_df['Weighted'].sum() / (merged_df['Length'].sum()/2)

        traffic_gp = (traffic_anualization/year_days) * merged_gp_traffic_df['Weighted'].sum() / (merged_gp_traffic_df['Length'].sum()/2)

        gp_ml = total_aadt + traffic_gp

        capture_total = total_aadt / gp_ml

        toll_AADT_mi = total_revenues / (year_days * total_aadt * (merged_df['Length'].sum()/2))
        
        output_df.loc[len(output_df)] = [year, total_revenues, total_transactions, total_aadt, toll_AADT_mi, capture_total]

    # Define full range of years to interpolate over
    full_years = pd.DataFrame({"Year": np.arange(int(output_df["Year"].min()), int(output_df["Year"].max()) + 1)})

    # Merge with original to create missing years with NaNs
    merged = pd.merge(full_years, output_df, on="Year", how="left")

    # Interpolate all numeric columns except "Year"
    interpolated = np.log(merged).interpolate(method="linear")

    interpolated = np.exp(interpolated)

    # Copy original column names
    original_columns = list(interpolated.columns)

    # Track how many columns we've inserted to adjust the index
    insert_count = 0

    # Start from index 1 to skip the first column (index 0)
    for i in range(1, len(original_columns)):
        col = original_columns[i]
        new_col_name = f"Var {col} (%)"
        insert_position = i + insert_count + 1  # Adjust position with insert_count
        interpolated[col] = interpolated[col].round(6)
        new_col_val = interpolated[col].pct_change()
        interpolated.insert(insert_position, new_col_name, new_col_val)
        insert_count += 1

    interpolated.to_csv(file_name)

    return interpolated

In [121]:
interpolated = generate_revenue_stream(first_model_df, f'{run_folder}/revenue_stream.csv')

interpolated

C:\Users\crisp\AppData\Local\Temp\ipykernel_23984\990158868.py:53: RuntimeWarning: invalid value encountered in scalar divide
  total_aadt = (traffic_anualization/year_days) * merged_df['Weighted'].sum() / (merged_df['Length'].sum()/2)
C:\Users\crisp\AppData\Local\Temp\ipykernel_23984\990158868.py:55: RuntimeWarning: invalid value encountered in scalar divide
  traffic_gp = (traffic_anualization/year_days) * merged_gp_traffic_df['Weighted'].sum() / (merged_gp_traffic_df['Length'].sum()/2)
C:\Users\crisp\AppData\Local\Temp\ipykernel_23984\990158868.py:53: RuntimeWarning: invalid value encountered in scalar divide
  total_aadt = (traffic_anualization/year_days) * merged_df['Weighted'].sum() / (merged_df['Length'].sum()/2)
C:\Users\crisp\AppData\Local\Temp\ipykernel_23984\990158868.py:55: RuntimeWarning: invalid value encountered in scalar divide
  traffic_gp = (traffic_anualization/year_days) * merged_gp_traffic_df['Weighted'].sum() / (merged_gp_traffic_df['Length'].sum()/2)
C:\Users\cri

,Year,Total Revenue,Var Total Revenue (%),Total Transactions,Var Total Transactions (%),AADT,Var AADT (%),Toll/AADT/mi,Var Toll/AADT/mi (%),Capture,Var Capture (%)
0,2032.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2033.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2034.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2035.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2036.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2037.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2038.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2039.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2040.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2041.0,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
